# Chapter 2: Working with Text Data

- Focus: prepare and clean text for language modeling
- Learn tokenization and build a small vocabulary
- Turn raw text into numeric inputs we can train on

Packages used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

## 2.1 Understanding word embeddings

- For LLMs, we treat tokens as vectors in a high-dimensional space
- These vectors encode relationships that the model can learn
- We often use 2D sketches to build intuition about a much larger space

## 2.2 Tokenizing text

- Tokenization splits text into units (words, punctuation, or subwords)
- These units become the inputs the model will work with
- I’ll use a public-domain text (The Verdict) as sample data

In [ ]:
import os

# Check for local file first (zero-copy policy compliance)
# The file should already exist in the notebook directory
file_path = "the-verdict.txt"

if not os.path.exists(file_path):
    # Fallback: download from public domain source if local file missing
    # Note: This is a public domain text by Edith Wharton
    import requests
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_path} from external source")
else:
    print(f"Using local file: {file_path}")


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

- Goal: turn the raw text into tokens a model can consume
- I’ll build a simple regex tokenizer first, then scale it to the full text
- Start by splitting on whitespace to see the baseline behavior

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- Whitespace-only splitting misses punctuation boundaries
- Next step: treat commas and periods as their own tokens

In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

- The split introduces empty strings
- Filter them out to keep only meaningful tokens

In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

- Expand the pattern to cover more punctuation (e.g., ?, !, quotes)
- This should produce cleaner, more realistic token streams

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

- The regex looks good enough for a first pass
- Now apply it to the full text loaded earlier

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

- Count the total number of tokens produced

In [ ]:
print(len(preprocessed))

## 2.3 Converting tokens into IDs

- Models need numbers, not strings
- Build a vocabulary mapping each unique token to an integer ID
- Start by listing unique tokens in a consistent order

In [ ]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)

print(vocab_size)

- Create the token → ID dictionary using a simple enumeration

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Peek at a handful of entries to sanity-check the mapping

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

- Next, I’ll wrap the logic into a small tokenizer class
- Then I’ll test it on a short sample string

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` converts text into token IDs
- `decode` converts token IDs back into text
- These IDs are what we’ll later feed into embeddings for the model

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

- Decode the IDs to verify the round-trip works

In [ ]:
tokenizer.decode(ids)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

## 2.4 Adding special context tokens

- It's useful to add some "special" tokens for unknown words and to denote the end of a text
- Some tokenizers use special tokens to help the LLM with additional context
- Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks the beginning of text
  - `[EOS]` (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
  - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
- `[UNK]` to represent words that are not included in the vocabulary

- Note that GPT-2 does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token to reduce complexity
- The `<|endoftext|>` is analogous to the `[EOS]` token mentioned above
- GPT also uses the `<|endoftext|>` for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)
- GPT-2 does not use an `<UNK>` token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section
- We use the `<|endoftext|>` tokens between two independent sources of text:
- Let's see what happens if we tokenize the following text:


In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)